<h1>Getting Started with PySC2</h1>
<p>For additional information, please refer to the official GitHub repository linked <a href="https://github.com/deepmind/pysc2">here</a>.</p>
<p>To get started with PySC2, first verify that you have the proper Python packages installed as well as the mini game maps linked in the PySC2 repository!</p>
<p>Please run the following cell to make sure that your computer has all of the proper software setup.</p>
<p>If you have everything installed properly, the running the following cell should open an instance of StarCraft with a randomly acting agent playing on the MoveToBeacon map:</p>
<div style="background-color:#300a24"><b><p style="color:white">python3 -m pysc2.bin.agent --map MoveToBeacon --agent pysc2.agents.random_agent.RandomAgent</p></b></div>

<h3>Actions</h3>
<p>Assuming all of that worked, we will continue by setting up some of the basic configuration for our StarCraft AI. In order to make our AI, we will create a Python class to represent our StarCraft agent. We will make our class inherit from the base agent which PySC2 provides.</p>

<p>Some things worth noting are that we define a list of default actions for our agent to use. This will be used to restrict which actions our agent is allowed to perform. To view the list of all valid actions our agent can perform, try entering the following command in a terminal:</p>
<div style="background-color:#300a24"><b><p style="color:white">python3 -m pysc2.bin.valid_actions --hide_specific</p></b></div>


<p>Running this command produces many lines of output giving you the numerical id's of various StarCraft actions our agent can perform. A smaller subset of actions has been selected with comments with their name so that our agent is not slowed down with too many actions to learn.<p>

In [ ]:
from pysc2.agents.base_agent import BaseAgent
from pysc2.lib import actions, features
import random
import numpy as np

default_actions = [
   0, #no_op                                              ()
   1, #move_camera                                        (1/minimap [64, 64])
#   2, #select_point                                       (6/select_point_act [4]; 0/screen [84, 84])
#   3, #select_rect                                        (7/select_add [2]; 0/screen [84, 84]; 2/screen2 [84, 84])
#   4, #select_control_group                               (4/control_group_act [5]; 5/control_group_id [10])
   5, #select_unit                                        (8/select_unit_act [4]; 9/select_unit_id [500])
#   6, #select_idle_worker                                 (10/select_worker [4])
   7, #select_army                                        (7/select_add [2])
 #  8, #select_warp_gates                                  (7/select_add [2])
 #  9, #select_larva                                       ()
 # 10, #unload                                             (12/unload_id [500])
 # 11, #build_queue                                        (11/build_queue_id [10])
 # 12, #Attack_screen                                      (3/queued [2]; 0/screen [84, 84])
 # 13, #Attack_minimap                                     (3/queued [2]; 1/minimap [64, 64])
 331, #Move_screen                                        (3/queued [2]; 0/screen [84, 84])
 332 #Move_minimap                                       (3/queued [2]; 1/minimap [64, 64])
]


_PLAYER_SELF = features.PlayerRelative.SELF
_PLAYER_NEUTRAL = features.PlayerRelative.NEUTRAL  # beacon/minerals
_PLAYER_ENEMY = features.PlayerRelative.ENEMY


#This represents the base interface for how our agent will work
#We separate this from the StarCraft II agent class so we can focus on the underlying RL
#implementation later...
class Brain:
    def __init__(self, race="T", actions = default_actions):
        self.race = race
        self.actions = actions
    #By default, our brain will just do nothing.
    #We will change this later...
    def step(self, obs):
        return 0, []
        
#        if 331 in obs.observation.available_actions:
#            player_relative = obs.observation.feature_screen.player_relative
#            beacon = _xy_locs(player_relative == _PLAYER_NEUTRAL)
#            if not beacon:
#                return 0, []
#            beacon_center = np.mean(beacon, axis=0).round()
#            return 331, [[0], beacon_center]
#        else:
#            return 7, [[0]]


#This represents the actual agent which will play StarCraft II
class MyAgent(BaseAgent):
    def __init__(self, brain = Brain()):
        super().__init__() #call parent constructor
        assert isinstance(brain, Brain)
        self.brain = brain
        
    def step(self, obs): #This function is called once per frame to give the AI observation data and return its action
        super().step(obs) #call parent base method
        action, params = self.brain.step(obs)
        return actions.FunctionCall(action, params)
        
agent = MyAgent()

<p>From here, we can test our our agent by calling the following cell. The first line exports our notbook code as a Python file. The second line actually runs our agent.</p>

<div style="background-color:#300a24"><b><p style="color:white">
jupyter nbconvert --to script PySC2_Basics<br>python3 -m pysc2.bin.agent --map MoveToBeacon --agent PySC2_Basics.MyAgent</p></b></div>

<p>To scale up our training performance, we will be using the Synchronous Actor Advantage Critic (A2C) reinforcement learning algorithm, which allows us to train our agent multiple times in parallel. Starter code is provided <a href="https://github.com/MG2033/A2C">here</a>. First, we will need to import some modules.<p>

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
UNIT_ELEMENTS = 7
MAXIMUM_CARGO = 10
MAXIMUM_BUILD_QUEUE = 10
MAXIMUM_MULTI_SELECT = 10
class StateNet:
    def __init__(self, scope, nonspatial_actions = len(default_actions),
                 resolution=84, channels=20, max_multi_select=MAXIMUM_MULTI_SELECT,
                 max_cargo=MAXIMUM_CARGO, max_build_queue=MAXIMUM_BUILD_QUEUE,
                 l2_scale=0.01, hidden_size=256):
        self.resolution = resolution
        self.variable_feature_sizes = {
            'multi_select' : max_multi_select,
            'cargo' :  max_cargo, 
            'build_queue' : max_build_queue
        }
        #The following assumes that we will stack our minimap and screen features (and they will have the same size)
        with tf.variable_scope('State-{}'.format(scope)):
            self.structured_observation = tf.placeholder(tf.float32, [None, 11], 'StructuredObservation')
            self.single_select = tf.placeholder(tf.float32, [None, 1, UNIT_ELEMENTS], 'SingleSelect')
            self.cargo = tf.placeholder(tf.float32, [None,  max_cargo, UNIT_ELEMENTS], 'Cargo')
            self.multi_select = tf.placeholder(tf.float32, [None, max_multi_select, UNIT_ELEMENTS], 'Multiselect')
            self.build_queue = tf.placeholder(tf.float32, [None,  max_build_queue, UNIT_ELEMENTS], 'BuildQueue')
            self.units = tf.concat([self.single_select,
                                    self.multi_select,
                                    self.cargo,
                                    self.build_queue], axis=1,
                                    name='Units')
            self.control_groups = tf.placeholder(tf.float32, [None, 10, 2], 'ControlGroups')
            self.available_actions = tf.placeholder(tf.float32, [None, nonspatial_actions], 'AvailableActions')
            self.used_actions = tf.placeholder(tf.float32, [None, nonspatial_actions], 'UsedActions')
            self.actions = tf.concat([self.available_actions,
                                      self.used_actions], axis=1,
                                      name='Actions')
            self.nonspatial_features = tf.concat([
                    self.structured_observation,
                    tf.reshape(self.units, [-1, UNIT_ELEMENTS * (1+sum(self.variable_feature_sizes.values()))]),
                    tf.reshape(self.control_groups, [-1, 20]),
                    tf.reshape(self.actions, [-1, 2 * nonspatial_actions])
                ], axis=1, name='NonspatialFeatures')
            
            self.spatial_features = tf.placeholder(tf.float32,
                                                   [None, resolution, resolution, channels],
                                                   'SpatialFeatures')
            self.conv1 = tf.layers.conv2d(inputs=self.spatial_features, filters=32,
                                          kernel_size=[5, 5],
                                          kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_scale),
                                          activation=tf.nn.relu, name='Convolutional1')
            self.max_pool1 = tf.layers.max_pooling2d(inputs=self.conv1, pool_size=[2, 2],
                                                     strides=2, name='Pool1')
            self.conv2 = tf.layers.conv2d(inputs=self.max_pool1, filters=64,
                                          kernel_size=[5, 5],
                                          kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_scale),
                                          activation=tf.nn.relu, name='Convolutional2')
            self.max_pool2 = tf.layers.max_pooling2d(inputs=self.conv2, pool_size=[2, 2],
                                                     strides=2, name='Pool2')
            self.max_pool2_flat = tf.reshape(self.max_pool2, [-1, 18 * 18 * 64], name='Pool2_Flattened')
            self.state_flattened = tf.concat([self.max_pool2_flat, self.nonspatial_features],
                                             1, name='StateFlattened')
            self.hidden_1 = tf.layers.dense(self.state_flattened, hidden_size, tf.nn.relu, name='Hidden1')
            self.output = self.hidden_1
            for variable_name, tensor in vars(self).items():
                if isinstance(tensor, tf.Tensor):
                    print('{}:\t({} Shape={})'.format(variable_name, tensor.name, tensor.shape))
            
tf.reset_default_graph()
test_state = StateNet('test')
global_state = StateNet('global')

In [ ]:
class QNet:
    def __init__(self, statenet, scope, usable_actions=default_actions):
        self.actions = usable_actions
        with tf.variable_scope(scope):
            self.action_probability_raw = tf.layers.dense(statenet.output,
                                                          len(self.actions),
                                                          tf.nn.relu,
                                                          name='ActionProbRaw')
            self.action_probability = tf.nn.softmax(self.action_probability_raw)
            print('Action probability shape:', self.action_probability.shape)
            self.arguments = {}
            for argument in actions.TYPES:
                self.arguments[argument.name] = [] 
                for dimension, size in enumerate(argument.sizes):
                    if size == 0: #set size for screen/minimap coordinates
                        size = 1
                        if argument.name in ['screen', 'screen2', 'minimap']:
                            size = statenet.resolution
                    argument_layer = tf.layers.dense(statenet.output, 
                                                     size, tf.nn.relu,
                                                     name='{}{}'.format(argument.name,
                                                                        dimension))
                    print('Argument {}[{}] Shape:{}'.format(argument.name, dimension, argument_layer.shape))
                    self.arguments[argument.name].append(argument_layer)
            self.value =  tf.layers.dense(statenet.output, 1, name='Value')
            print('{} Shape: {}'.format(self.value.name, self.value.shape))
        
test_q = QNet(test_state, 'test')
global_q = QNet(global_state, 'global')

In [ ]:
entropy_factor = 0.1
value_factor = 0.5
gradient_norm_factor = 40
default_optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
class QTrainer:
    def __init__(self, statenet, qnet, scope, optimizer=default_optimizer):
        with tf.variable_scope('QTrainer-{}'.format(scope)):
            self.action = tf.placeholder(tf.int32, [None], name='Action_Placeholder')
            self.action_one_hot = tf.one_hot(self.action, len(qnet.actions), name='Actions_One_Hot')
            self.entropy_base = -tf.reduce_sum(qnet.action_probability * tf.log(
                    tf.clip_by_value(qnet.action_probability, 1e-20, 1)
                ), [1]
            )
            self.arguments = {} #placeholders for argument training
            self.arguments_one_hot = {}
            self.arguments_entropy = {}
            for argument in actions.TYPES:
                self.arguments[argument.name] = [] 
                self.arguments_one_hot[argument.name] =[]
                self.arguments_entropy[argument.name] = []
                for dimension, size in enumerate(argument.sizes):
                    argument_placeholder = tf.placeholder(tf.int32,[None],
                                                          name='{}{}_Placeholder'.format(argument.name,
                                                                                         dimension))
                    self.arguments[argument.name].append(argument_placeholder)
                    argument_one_hot = tf.one_hot(argument_placeholder,
                                                  qnet.arguments[argument.name][dimension].shape[1],
                                                  dtype=tf.float32,
                                                  name='{}{}_One_Hot'.format(argument.name,
                                                                                 dimension))
                    self.arguments_one_hot[argument.name].append(argument_one_hot)
                    print('{} Shape:{}'.format(argument_one_hot.name,
                                                        argument_one_hot.shape))
                    
                    argument_entropy = -tf.reduce_sum(
                        qnet.arguments[argument.name][dimension] * tf.log(
                            tf.clip_by_value(qnet.arguments[argument.name][dimension], 1e-20, 1)
                        ), [1]
                    )
                    self.arguments_entropy[argument.name].append(argument_entropy)
                    self.entropy_base += argument_entropy
            self.target_value = tf.placeholder(tf.float32, shape=[None], name='Target_Value')
            #Multiplying our QNet's action probabilities by the one hot action tensor
            self.value_loss = qnet.value - self.target_value
            self.loss = self.value_loss * value_factor + self.entropy_base * entropy_factor
            local_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)
            self.gradients = tf.gradients(self.loss, local_vars)
            self.var_norms = tf.global_norm(local_vars)
            grads, self.grad_norms = tf.clip_by_global_norm(self.gradients, gradient_norm_factor)

            # Apply local gradients to global network
            global_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)#'global')
            self.apply_grads = optimizer.apply_gradients(zip(grads,global_vars))
            
test_qtrainer = QTrainer(test_state, test_q, 'test')

In [ ]:
class RLBrain(Brain):
    def __init__(self, name, sess=None, race="T", actions = default_actions):
        super().__init__()
        self.state = StateNet(name, nonspatial_actions=len(actions))
        self.q = QNet(self.state, name, usable_actions=actions)
        self.q_trainer = QTrainer(self.state, self.q, name)
        self.sess = sess
    #By default, our brain will just do nothing.
    #We will change this later...
    def step(self, obs):
        #formatting/processing our observation to feed into state/q nets
        #determine our action probabilities
        #select action randomly with probabilities
        #for each required  argument:
            #determine argument value probabilities
            #select argument value randomly with probabilities
        #return action id, arguments
        return 0, []
    
    def process_observations(self, observation):
        # is episode over?
        episode_end = (observation.step_type == environment.StepType.LAST)
        # reward
        reward = observation.reward #scalar?
        # features
        features = observation.observation
        # the shapes of some features depend on the state (eg. shape of multi_select depends on number of units)
        # since tf requires fixed input shapes, we set a maximum size then pad the input if it falls short
        processed_features = {}
        for feature_label, feature in observation.observation.items():
            if feature_label in ['available_actions', 'last_actions']:
                actions = np.zeros(len(self.actions))
                for i, action in enumerate(self.actions):
                    if action in feature:
                        actions[i] = 1
                feature = actions
            if feature_label in ['minimap', 'screen']:
                feature = np.stack(feature, axis=2)
            elif feature_label in ['single_select', 'multi_select', 'cargo', 'build_queue']:
                if feature_label in self.state.variable_feature_sizes:
                    padding = np.zeros((self.state.variable_feature_sizes[feature_label] - len(feature), UNIT_ELEMENTS))
                    feature = np.concatenate(feature, padding)
                feature = feature.reshape(-1, UNIT_ELEMENTS)
            elif feature_label == 'structured':
                
            else:
                continue
            processed_features[feature_label] = np.expand_dims(feature, axis=0)
        return reward, processed_features, episode_end


In [ ]:
tf.reset_default_graph()
brain = RLBrain('test')

In [ ]:
class SimpleNet:
    def __init__(self, scope, statenet, resolution=84):
        self.statenet = statenet
        self.input = statenet.output
        self.hidden_layer = tf.layers.dense(self.input, 256, activation=tf.nn.relu)
        self.raw_output = tf.layers.dense(self.hidden_layer, 2)
        self.clipped_output = tf.clip_by_value(self.raw_output, 0, resolution-1)
        self.output = tf.cast(self.clipped_output, tf.int32)
        for variable_name, tensor in vars(self).items():
            if isinstance(tensor, tf.Tensor):
                print('{}:\t({} Shape={})'.format(variable_name, tensor.name, tensor.shape))
        
        
        
class SimpleBrain(Brain):
    def  __init__(self, scope, race="T", actions = default_actions):
        super().__init__(race, actions)
        self.statenet = StateNet(scope)
        self.simplenet = SimpleNet(scope, self.statenet)
        self.sess = tf.Session()
        self.feature_placeholders = {
            'available_actions' : self.statenet.available_actions,
            'last_actions' : self.statenet.used_actions,
            'cargo' : self.statenet.cargo,
            'multi_select' : self.statenet.multi_select,
            'single_select' : self.statenet.single_select,
            'build_queue' : self.statenet.build_queue,
            'structured' : self.statenet.structured_observation,
            'control_groups' : self.statenet.control_groups,
            'multi_select' : self.statenet.multi_select,
        }
        
        
    def step(self, obs):
        if 331 in obs.observations.available_actions:
            reward, feed_dict, episode_end = self.proccess_observations(obs)
            outputs = sess.run(simplenet.output, feed_dict)
            
    def process_observations(self, observation):
        # is episode over?
        episode_end = (observation.step_type == environment.StepType.LAST)
        # reward
        reward = observation.reward #scalar?
        # features
        features = observation.observation
        # the shapes of some features depend on the state (eg. shape of multi_select depends on number of units)
        # since tf requires fixed input shapes, we set a maximum size then pad the input if it falls short
        processed_features = {}
        for feature_label, feature in observation.observation.items():
            if feature_label in ['available_actions', 'last_actions']:
                actions = np.zeros(len(self.actions))
                for i, action in enumerate(self.actions):
                    if action in feature:
                        actions[i] = 1
                feature = actions
            if feature_label in ['minimap', 'screen']:
                feature = np.stack(feature, axis=2)
            elif feature_label in ['single_select', 'multi_select', 'cargo', 'build_queue']:
                if feature_label in self.state.variable_feature_sizes:
                    padding = np.zeros((self.state.variable_feature_sizes[feature_label] - len(feature), UNIT_ELEMENTS))
                    feature = np.concatenate(feature, padding)
                feature = feature.reshape(-1, UNIT_ELEMENTS)
            elif feature_label == 'structured':
                pass
            else:
                continue
            placeholder = self.feature_placeholders[feature_label]
            processed_features[placeholder] = np.expand_dims(feature, axis=0)
        return reward, processed_features, episode_end


In [ ]:
from pysc2.bin.agent import main
from absl import app
import tensorflow as tf
tf.reset_default_graph()

#point_flag.DEFINE_point("feature_screen_size", "84",
#                        "Resolution for screen feature layers.")
#point_flag.DEFINE_point("feature_minimap_size", "64",
#"Resolution for minimap feature layers.")
args = [ None, '--map', 'MoveToBeacon', '--agent=MyAgent.SimpleAgent', '--feature_minimap_size', '84']
app.run(main, args)

/home/student/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
I0713 17:51:51.563463 140319881676544 sc_process.py:110] Launching SC2: /home/student/StarCraftII/Versions/Base59877/SC2_x64 -listen 127.0.0.1 -port 21508 -dataDir /home/student/StarCraftII/ -tempDir /tmp/sc-vexvs3bd/ -displayMode 0 -windowwidth 640 -windowheight 480 -windowx 50 -windowy 50 -eglpath libEGL.so
I0713 17:51:51.573115 140319881676544 remote_controller.py:127] Connecting to: ws://127.0.0.1:21508/sc2api, attempt: 0, running: True
I0713 17:51:52.577271 140319881676544 remote_controller.py:127] Connecting to: ws://127.0.0.1:21508/sc2api, attempt: 1, running: True
I0713 17:51:53.580788 140319881676544 remote_controller.py:127] Connecting to: ws://127.0.0.1:21508/sc2a

structured_observation:	(State-test/StructuredObservation:0 Shape=(?, 11))
single_select:	(State-test/SingleSelect:0 Shape=(?, 1, 7))
cargo:	(State-test/Cargo:0 Shape=(?, 10, 7))
multi_select:	(State-test/Multiselect:0 Shape=(?, 10, 7))
build_queue:	(State-test/BuildQueue:0 Shape=(?, 10, 7))
units:	(State-test/Units:0 Shape=(?, 31, 7))
control_groups:	(State-test/ControlGroups:0 Shape=(?, 10, 2))
available_actions:	(State-test/AvailableActions:0 Shape=(?, 6))
used_actions:	(State-test/UsedActions:0 Shape=(?, 6))
actions:	(State-test/Actions:0 Shape=(?, 12))
nonspatial_features:	(State-test/NonspatialFeatures:0 Shape=(?, 260))
screen_features:	(State-test/ScreenFeatures:0 Shape=(?, 17, 84, 84))
minimap_features:	(State-test/MinimapFeatures:0 Shape=(?, 7, 84, 84))
spatial_features:	(State-test/transpose:0 Shape=(?, 84, 84, 24))
conv1:	(State-test/Convolutional1/Relu:0 Shape=(?, 80, 80, 32))
max_pool1:	(State-test/Pool1/MaxPool:0 Shape=(?, 40, 40, 32))
conv2:	(State-test/Convolutional2/Re

I0713 17:51:59.068670 140319881676544 sc2_env.py:433] Starting episode: 1


   0/no_op                                              ()
   1/move_camera                                        (1/minimap [84, 84])
   2/select_point                                       (6/select_point_act [4]; 0/screen [84, 84])
   3/select_rect                                        (7/select_add [2]; 0/screen [84, 84]; 2/screen2 [84, 84])
   4/select_control_group                               (4/control_group_act [5]; 5/control_group_id [10])
 453/Stop_quick                                         (3/queued [2])
 451/Smart_screen                                       (3/queued [2]; 0/screen [84, 84])
   7/select_army                                        (7/select_add [2])
 452/Smart_minimap                                      (3/queued [2]; 1/minimap [84, 84])
 331/Move_screen                                        (3/queued [2]; 0/screen [84, 84])
 332/Move_minimap                                       (3/queued [2]; 1/minimap [84, 84])
 333/Patrol_screen                 

I0713 17:52:01.500394 140319881676544 sc2_env.py:510] Episode 1 finished after 1920 game steps. Outcome: [1], reward: [0], score: [1]
I0713 17:52:01.523944 140319881676544 sc2_env.py:433] Starting episode: 2
I0713 17:52:03.431560 140319881676544 sc2_env.py:510] Episode 2 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 17:52:03.448613 140319881676544 sc2_env.py:433] Starting episode: 3
I0713 17:52:05.319805 140319881676544 sc2_env.py:510] Episode 3 finished after 1920 game steps. Outcome: [1], reward: [0], score: [1]
I0713 17:52:05.345485 140319881676544 sc2_env.py:433] Starting episode: 4
I0713 17:52:07.033756 140319881676544 sc2_env.py:510] Episode 4 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 17:52:07.061019 140319881676544 sc2_env.py:433] Starting episode: 5
I0713 17:52:08.823863 140319881676544 sc2_env.py:510] Episode 5 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 17:52:08.853568 140319881676

...

I0713 18:43:56.589128 140319881676544 sc2_env.py:433] Starting episode: 1786
I0713 18:43:58.274100 140319881676544 sc2_env.py:510] Episode 1786 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:43:58.300422 140319881676544 sc2_env.py:433] Starting episode: 1787
I0713 18:43:59.988040 140319881676544 sc2_env.py:510] Episode 1787 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:44:00.006966 140319881676544 sc2_env.py:433] Starting episode: 1788
I0713 18:44:01.685818 140319881676544 sc2_env.py:510] Episode 1788 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:44:01.704048 140319881676544 sc2_env.py:433] Starting episode: 1789
I0713 18:44:03.422194 140319881676544 sc2_env.py:510] Episode 1789 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:44:03.444327 140319881676544 sc2_env.py:433] Starting episode: 1790
I0713 18:44:05.157318 140319881676544 sc2_env.py:510] Episode 1790 

I0713 18:45:04.156406 140319881676544 sc2_env.py:510] Episode 1824 finished after 1920 game steps. Outcome: [1], reward: [0], score: [1]
I0713 18:45:04.180105 140319881676544 sc2_env.py:433] Starting episode: 1825
I0713 18:45:05.947177 140319881676544 sc2_env.py:510] Episode 1825 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:45:05.965307 140319881676544 sc2_env.py:433] Starting episode: 1826
I0713 18:45:07.687135 140319881676544 sc2_env.py:510] Episode 1826 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:45:07.731156 140319881676544 sc2_env.py:433] Starting episode: 1827
I0713 18:45:09.510792 140319881676544 sc2_env.py:510] Episode 1827 finished after 1920 game steps. Outcome: [1], reward: [0], score: [3]
I0713 18:45:09.533904 140319881676544 sc2_env.py:433] Starting episode: 1828
I0713 18:45:11.263825 140319881676544 sc2_env.py:510] Episode 1828 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 1

I0713 18:46:10.157275 140319881676544 sc2_env.py:433] Starting episode: 1863
I0713 18:46:11.882677 140319881676544 sc2_env.py:510] Episode 1863 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:46:11.908336 140319881676544 sc2_env.py:433] Starting episode: 1864
I0713 18:46:13.629604 140319881676544 sc2_env.py:510] Episode 1864 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:46:13.653897 140319881676544 sc2_env.py:433] Starting episode: 1865
I0713 18:46:15.367434 140319881676544 sc2_env.py:510] Episode 1865 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:46:15.393677 140319881676544 sc2_env.py:433] Starting episode: 1866
I0713 18:46:17.093823 140319881676544 sc2_env.py:510] Episode 1866 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:46:17.111919 140319881676544 sc2_env.py:433] Starting episode: 1867
I0713 18:46:18.823244 140319881676544 sc2_env.py:510] Episode 1867 

I0713 18:47:17.741657 140319881676544 sc2_env.py:510] Episode 1901 finished after 1920 game steps. Outcome: [1], reward: [0], score: [2]
I0713 18:47:17.765898 140319881676544 sc2_env.py:433] Starting episode: 1902
I0713 18:47:19.446322 140319881676544 sc2_env.py:510] Episode 1902 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:47:19.466696 140319881676544 sc2_env.py:433] Starting episode: 1903
I0713 18:47:21.166786 140319881676544 sc2_env.py:510] Episode 1903 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:47:21.184074 140319881676544 sc2_env.py:433] Starting episode: 1904
I0713 18:47:22.873818 140319881676544 sc2_env.py:510] Episode 1904 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:47:22.901614 140319881676544 sc2_env.py:433] Starting episode: 1905
I0713 18:47:24.646659 140319881676544 sc2_env.py:510] Episode 1905 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 1

I0713 18:48:25.559965 140319881676544 sc2_env.py:433] Starting episode: 1940
I0713 18:48:27.264991 140319881676544 sc2_env.py:510] Episode 1940 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:48:27.292201 140319881676544 sc2_env.py:433] Starting episode: 1941
I0713 18:48:29.021601 140319881676544 sc2_env.py:510] Episode 1941 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:48:29.040311 140319881676544 sc2_env.py:433] Starting episode: 1942
I0713 18:48:30.732726 140319881676544 sc2_env.py:510] Episode 1942 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:48:30.764600 140319881676544 sc2_env.py:433] Starting episode: 1943
I0713 18:48:32.463329 140319881676544 sc2_env.py:510] Episode 1943 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I0713 18:48:32.482289 140319881676544 sc2_env.py:433] Starting episode: 1944
I0713 18:48:34.179845 140319881676544 sc2_env.py:510] Episode 1944 